Creating a Unigram Tokenizer on the version 6 of the corpora
----------------------------------
We added new sentences to extracted sentences from the book <i style="color: cyan">Grammaire de Wolof Moderne</i> by Pathé Diagne plus the original corpora to obtain the sixth version of it.

The process is almost the same as in [processing_4](text_processing4.ipynb) excepted that we will create another custom dataset for the custom transformer model and identify with a box plot the range of the maximum length of the sequences in order to tune the `max_len` parameter provided to the custom dataset.

Let us import the necessary libraries.

In [11]:
# for creating the tokenizer
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    normalizers
)

# for importing and manipulating the sentences
import pandas as pd
import random

# for plotting the box plot of the sequence lengths
import plotly.express as px

# for loading sentences with the custom dataset
from torch.utils.data import DataLoader

#### Load dataset and create generator

We will create one tokenizer for both of the French and Wolof corpora because the `T5` model understand only one embedding layer. So we must create one generator for both of the French and Wolof corpora. 

In [12]:
# load sentences
sentences = pd.read_csv("data/extractions/new_data/corpora_v6.csv")

# initialize a batch size
BATCH_SIZE = 400

# create generators (for the corpora)
def generate_sents():
    
    # recuperate the sentences
    french_sents = sentences['french'].to_list() 
    
    wolof_sents = sentences['wolof'].to_list() 
    
    sents = french_sents + wolof_sents
    
    for i in range(1, len(sents), BATCH_SIZE):
        
        yield sents[i:i+BATCH_SIZE]

#### Initialize the tokenizer

In [13]:
tokenizer = Tokenizer(models.Unigram())

#### Add normalizer

In [14]:
tokenizer.normalizer = normalizers.Replace(" {2,}", " ")

#### Configure the pre-tokenizers

We will use the Metaspace pre-tokenizer which separates the words considering the spaces between them. It will replace the space by a character (by default the underscore "_").

In [15]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

#### Initialize the trainers

We will provide all of the necessary special tokens to the Trainer. 

**Notice that a sentence can be a group of words separated by ending marks and not only one group of words. Then we can, for example, tokenize the following sentences**: `<sep>sentence1.sentence2.sentence3<cls>` **or** `<sep>sentence1.<sep>sentence2.<cls>`. **But, the second sentence is composed of two separate groups. Then the two sentences will have different type ids.** 

In [16]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]

In [17]:
trainer = trainers.UnigramTrainer(special_tokens=special_tokens, unk_token = "<unk>", vocab_size=20000) # let us take the default vocab size

#### Train the tokenizer

In [18]:
tokenizer.train_from_iterator(generate_sents(), trainer)

Let us print the vocab size.

In [19]:
print(f"Number of tokens: {tokenizer.get_vocab_size()}")

Number of tokens: 19801


#### Initialize the post-processor

We can not need the TemplateProcessor to train our corpora in a Sequence To Sequence model, but we will add it to our tokenizer. We can use it for another type of model. 

In [20]:
# let us recuperate the sep and cls ids
cls_token_id = tokenizer.token_to_id("<cls>")

sep_token_id = tokenizer.token_to_id("<sep>")

print(cls_token_id, sep_token_id)

0 1


In [21]:
# Initialize the post processor
tokenizer.post_process = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)]
)

#### Initialize the decoder

In [22]:
tokenizer.decoder = decoders.Metaspace()

#### Save the tokenizer

In [23]:
tokenizer.save("wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v5.json")

#### Make a little example

Let us recuperate random sentences from the corpora and tokenize them.

In [24]:
random.seed(200)

french_sentence = random.choice(sentences['french']) 

wolof_sentence = random.choice(sentences['wolof']) 

# For the T5
french_sentence_t5 = french_sentence + "</s>"
wolof_sentence_t5 = wolof_sentence + "</s>"

# For the GPT2 (only example for french to wolof translation)
fr_sentence_gpt2 = "<s>" + french_sentence + "<sep>"
wf_sentence_gpt2 = wolof_sentence + "</s>"

In [25]:
# print the french sentence
french_sentence

"Mais à la fin de l'été 1968, encerclée, décimée par les troupes fédérales sous le commandement du général Benjamin Adekunle, surnommé pour sa cruauté le « Scorpion noir », l'armée biafraise capitule."

In [26]:
# print the wolof sentence
wolof_sentence

'Dafa aldànke foofa yépp.'

In [27]:
french_encoding_t5 = tokenizer.encode(french_sentence_t5)

print("French tokens t5")
print(french_encoding_t5.tokens)

print("French ids t5")
print(french_encoding_t5.ids)

French tokens t5
['▁Mais', '▁à', '▁la', '▁fin', '▁de', "▁l'", 'été', '▁1968', ',', '▁encerclé', 'e,', '▁déc', 'i', 'mée', '▁par', '▁les', '▁troupe', 's', '▁fédérale', 's', '▁sous', '▁le', '▁commandement', '▁du', '▁général', '▁Ben', 'j', 'amin', '▁Adekunle', ',', '▁sur', 'nommé', '▁pour', '▁sa', '▁cru', 'auté', '▁le', '▁', '«', '▁S', 'corpion', '▁noir', '▁»,', "▁l'armée", '▁biafrais', 'e', '▁capit', 'ule', '.', '</s>']
French ids t5
[628, 25, 10, 618, 11, 179, 2818, 6101, 9, 8302, 234, 5668, 24, 14211, 100, 33, 11956, 12, 13355, 12, 527, 17, 11548, 35, 4818, 915, 557, 9401, 4975, 9, 83, 15731, 51, 50, 3151, 6715, 17, 8, 231, 308, 16668, 956, 1283, 1989, 5011, 13, 10249, 7516, 7, 6]


In [28]:
wolof_encoding_t5 = tokenizer.encode(wolof_sentence_t5)

print("Wolof tokens t5")
print(wolof_encoding_t5.tokens)

print("Wolof ids t5")
print(wolof_encoding_t5.ids)

Wolof tokens t5
['▁Dafa', '▁', 'al', 'dànk', 'e', '▁foofa', '▁yépp', '.', '</s>']
Wolof ids t5
[64, 8, 88, 9542, 13, 2433, 264, 7, 6]


In [29]:
fr_encoding_gpt2 = tokenizer.encode(fr_sentence_gpt2)

print("French tokens gpt2")
print(fr_encoding_gpt2.tokens)

print("Wolof ids gpt2")
print(fr_encoding_gpt2.ids)

French tokens gpt2
['<s>', '▁Mais', '▁à', '▁la', '▁fin', '▁de', "▁l'", 'été', '▁1968', ',', '▁encerclé', 'e,', '▁déc', 'i', 'mée', '▁par', '▁les', '▁troupe', 's', '▁fédérale', 's', '▁sous', '▁le', '▁commandement', '▁du', '▁général', '▁Ben', 'j', 'amin', '▁Adekunle', ',', '▁sur', 'nommé', '▁pour', '▁sa', '▁cru', 'auté', '▁le', '▁', '«', '▁S', 'corpion', '▁noir', '▁»,', "▁l'armée", '▁biafrais', 'e', '▁capit', 'ule', '.', '<sep>']
Wolof ids gpt2
[5, 628, 25, 10, 618, 11, 179, 2818, 6101, 9, 8302, 234, 5668, 24, 14211, 100, 33, 11956, 12, 13355, 12, 527, 17, 11548, 35, 4818, 915, 557, 9401, 4975, 9, 83, 15731, 51, 50, 3151, 6715, 17, 8, 231, 308, 16668, 956, 1283, 1989, 5011, 13, 10249, 7516, 7, 1]


In [30]:
wf_encoding_gpt2 = tokenizer.encode(wf_sentence_gpt2)

print("Wolof tokens gpt2")
print(wf_encoding_gpt2.tokens)

print("Wolof ids gpt2")
print(wf_encoding_gpt2.ids)

Wolof tokens gpt2
['▁Dafa', '▁', 'al', 'dànk', 'e', '▁foofa', '▁yépp', '.', '</s>']
Wolof ids gpt2
[64, 8, 88, 9542, 13, 2433, 264, 7, 6]


#### Creating the T5 custom dataset for the new sentences

We have two possibilities to use the tokenizer for fine-tuning a T5 model. 

- We can use the `PreTrainedTokenizerFast` class for which we will provide the different special tokens.

In [31]:
# from transformers import PreTrainedTokenizerFast

# wrapped_tokenizer1 = PreTrainedTokenizerFast(
#     tokenizer_object=tokenizer,
#     bos_token="<s>",
#     eos_token="</s>",
#     unk_token="<unk>",
#     pad_token="<pad>",
#     cls_token="<cls>",
#     sep_token="<sep>",
#     mask_token="<mask>",
#     padding_side="left",
# )

- Or give directly the tokenizer to the `T5TokenizerFast` class.

In [32]:
from transformers import T5TokenizerFast

wrapped_tokenizer1 = T5TokenizerFast(
    tokenizer_object=tokenizer,
    bos_token = "<s>",
    sep_token = "<sep>"
)

In [33]:
wrapped_tokenizer1.convert_tokens_to_ids(['<s>', "<sep>"])

[5, 1]

Let us give them the sentences that we use as example. 

In [34]:
fr_encoding_t5 = wrapped_tokenizer1(french_sentence_t5, max_length=15, padding='max_length', truncation=True)

fr_encoding_t5

{'input_ids': [628, 25, 10, 618, 11, 179, 2818, 6101, 9, 8302, 234, 5668, 24, 14211, 100], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [35]:
wf_encoding_t5 = wrapped_tokenizer1(wolof_sentence_t5, max_length=15, padding='max_length', truncation=True)

wf_encoding_t5

{'input_ids': [64, 8, 88, 9542, 13, 2433, 264, 7, 6, 3, 3, 3, 3, 3, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [36]:
fr_encoding_gpt2 = wrapped_tokenizer1(fr_sentence_gpt2, max_length=15, padding='max_length', truncation=True)

fr_encoding_gpt2

{'input_ids': [5, 628, 25, 10, 618, 11, 179, 2818, 6101, 9, 8302, 234, 5668, 24, 14211], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [37]:
wf_encoding_gpt2 = wrapped_tokenizer1(wf_sentence_gpt2, max_length=15, padding='max_length', truncation=True)

wf_encoding_gpt2

{'input_ids': [64, 8, 88, 9542, 13, 2433, 264, 7, 6, 3, 3, 3, 3, 3, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

Let us decode the wolof sentence.

In [38]:
wrapped_tokenizer1.decode(fr_encoding_t5.input_ids + wf_encoding_t5.input_ids, skip_special_tokens=True)

"Mais à la fin de l'été 1968, encerclée, décimée par Dafa aldànke foofa yépp."

We can see that the `T5Tokenizer` add padding to the right side of the sequence while the `PretrainedTokenizer` add the padding to the left side. We can change the padding side from the settings. But, for the next steps, let us directly use the `T5Tokenizer`.

**Note that we can augment the sentences when generating them like we did when using the `GPT2 model`.** See the following notebook, [augmentation](text_augmentation.ipynb), for discussion on the augmentation method that we will use. And for a more clear explanation of the augmentation methods in NLP tasks and training, look at the following article from the web [augment_or_not](https://direct.mit.edu/coli/article/48/1/5/108844/To-Augment-or-Not-to-Augment-A-Comparative-Study).

Let us verify, before creating the custom dataset, the max length that we can get from the corpora' tokens without considering the augmentation. We must for that trace the box plot of the lengths and identify the range in which we will sample the max length of the sequences.

In [39]:
length = []

for sent in sentences['french'].to_list() + sentences['wolof'].to_list():
    
    len_ids = len(wrapped_tokenizer1(sent).input_ids)
    
    length.append(len_ids)

        

In [40]:
fig = px.box(x = length)

fig.update_layout({'xaxis': {'title': 'length'}})

The upper fence is of **21** and the max length is equal to **263**. Then we will test any value between the two. 

But considering the augmentation we can obtain more than the value that we will take because it will add modifications on the words and then it can recognize only parts of them and divide them in multiple other tokens. We will add to the max length the fifth of it. 

It is time to create our custom dataset for the t5 model. It can be also used for the GPT2 model: See later the test.

Signature:
```python
class T5SentenceDataset(Dataset):

    def __init__(
        self,
        data_path: str, 
        tokenizer: PreTrainedTokenizerFast
        corpus_1: str = "french",
        corpus_2: str = "wolof",
        max_len: int = 21,
        cp1_truncation: bool = False,
        cp2_truncation: bool = False,
        file_sep: str = ",",
        cp1_transformer: Union[TransformerSequences, None] = None,
        cp2_transformer: Union[TransformerSequences, None] = None,
        **kwargs):

        pass
```

In [41]:
%%writefile wolof-translate/wolof_translate/data/dataset_v4.py
from wolof_translate.utils.sent_transformers import TransformerSequences
from transformers import PreTrainedTokenizerFast
from torch.utils.data import Dataset
from typing import *
import pandas as pd
import torch
import re

class T5SentenceDataset(Dataset):

    def __init__(
        self,
        data_path: str, 
        tokenizer: PreTrainedTokenizerFast,
        corpus_1: str = "french",
        corpus_2: str = "wolof",
        max_len: int = 21,
        truncation: bool = False,
        file_sep: str = ",",
        cp1_transformer: Union[TransformerSequences, None] = None,
        cp2_transformer: Union[TransformerSequences, None] = None,
        decoder_only: bool = False,
        **kwargs):
        
        # let us recuperate the data frame
        self.__sentences = pd.read_csv(data_path, sep=file_sep, **kwargs)
        
        # let us recuperate the tokenizer
        self.tokenizer = tokenizer
        
        # recuperate the first corpus' sentences
        self.sentences_1 = self.__sentences[corpus_1].to_list()
        
        # recuperate the second corpus' sentences
        self.sentences_2 = self.__sentences[corpus_2].to_list()
        
        # recuperate the length
        self.length = len(self.sentences_1)
        
        # let us recuperate the max len
        self.max_len = max_len + max_len // 5
        
        # let us recuperate the truncation argument
        self.truncation = truncation
        
        # let us initialize the transformer
        self.cp1_transformer = cp1_transformer
        
        self.cp2_transformer = cp2_transformer
        
        # get the type of hugging face model
        self.decoder_only = decoder_only
        
    def __getitem__(self, index):
        """Recuperate ids and attention masks of sentences at index

        Args:
            index (int): The index of the sentences to recuperate

        Returns:
            tuple: The `sentence to translate' ids`, `the attention mask of the sentence to translate`
            `the labels' ids`
        """
        sentence_1 = self.sentences_1[index]
        
        sentence_2 = self.sentences_2[index]
        
        # apply transformers if necessary
        if not self.cp1_transformer is None:
            
            sentence_1 = self.cp1_transformer(sentence_1)[0]
        
        if not self.cp2_transformer is None:
            
            sentence_2 = self.cp2_transformer(sentence_2)[0]
        
        sentence_1 = sentence_1 + self.tokenizer.eos_token
        
        sentence_2 = sentence_2 + self.tokenizer.eos_token
        
        # let us encode the sentences (we provide the second sentence as labels to the tokenizer)
        data = self.tokenizer(
            sentence_1,
            truncation=self.truncation,
            max_length=self.max_len, 
            padding='max_length', 
            return_tensors="pt",
            text_target=sentence_2)
        
        return data.input_ids.squeeze(0), data.attention_mask.squeeze(0), data.labels.squeeze(0)
        
    def __len__(self):
        
        return self.length
    
    def decode(self, labels: torch.Tensor):
        
        if labels.ndim < 2:
            
            labels = labels.unsqueeze(0)

        sentences = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        return sentences


class SentenceDataset(T5SentenceDataset):

    def __init__(
        self,
        data_path: str, 
        tokenizer: PreTrainedTokenizerFast,
        corpus_1: str = "french",
        corpus_2: str = "wolof",
        max_len: int = 52,
        truncation: bool = False,
        file_sep: str = ",",
        cp1_transformer: Union[TransformerSequences, None] = None,
        cp2_transformer: Union[TransformerSequences, None] = None,
        **kwargs):
        
        super().__init__(data_path, 
                        tokenizer,
                        corpus_1,
                        corpus_2,
                        max_len,
                        truncation,
                        file_sep,
                        cp1_transformer,
                        cp2_transformer,
                        **kwargs)
        
    def __getitem__(self, index):
        """Recuperate ids and attention masks of sentences at index

        Args:
            index (int): The index of the sentences to recuperate

        Returns:
            tuple: The `sentence to translate' ids`, `the attention mask of the sentence to translate`
            `the labels' ids`
        """
        sentence_1 = self.sentences_1[index]
        
        sentence_2 = self.sentences_2[index]
        
        # apply transformers if necessary
        if not self.cp1_transformer is None:
            
            sentence_1 = self.cp1_transformer(sentence_1)[0] 
        
        if not self.cp2_transformer is None:
            
            sentence_2 = self.cp2_transformer(sentence_2)[0]
        
        sentence_1 = sentence_1 + self.tokenizer.eos_token if not self.decoder_only else\
            self.tokenizer.bos_token + sentence_1 + self.tokenizer.sep_token
        
        sentence_2 = sentence_2 + self.tokenizer.eos_token
        
        # let us encode the sentences (we provide the second sentence as labels to the tokenizer)
        data = self.tokenizer(
            sentence_1,
            truncation=self.truncation,
            max_length=self.max_len, 
            padding='max_length', 
            return_tensors="pt")
        
        # let us encode the sentences (we provide the second sentence as labels to the tokenizer)
        labels = self.tokenizer(
            sentence_2,
            truncation=self.truncation,
            max_length=self.max_len, 
            padding='max_length', 
            return_tensors="pt")
        
        return (data.input_ids.squeeze(0),
                data.attention_mask.squeeze(0), 
                labels.input_ids.squeeze(0),
                labels.attention_mask.squeeze(0))
    
    def decode(self, preds: torch.Tensor):
        
        if not self.decoder_only:
            
            sentences = super().decode(preds)
        
        else:
            
            preds = self.tokenizer.batch_decode(preds)
            
            sentences = []
            
            for pred in preds:
                
                sentences.append(re.findall('<sep>(.*)', pred)[-1]\
                    .replace(self.tokenizer.pad_token, "")\
                        .replace(self.tokenizer.eos_token, "")\
                            .replace(self.tokenizer.sep_token, "")\
                                .replace(self.tokenizer.bos_token, "").strip())

        return sentences
        
        
        

Overwriting wolof-translate/wolof_translate/data/dataset_v4.py


In [42]:
%run wolof-translate/wolof_translate/data/dataset_v4.py

Let us generate some data with their masks and decode the labels.

**Note that we will use, when training the `T5 model`, train and test sets and not directly the full dataset**

In [43]:
# Initialize our custom dataset
dataset = SentenceDataset("data/extractions/new_data/corpora_v6.csv", wrapped_tokenizer1, truncation=True)

In [44]:
generator = torch.manual_seed(5)
input_ids, input_mask, labels, _ = next(iter(DataLoader(dataset, 10, shuffle=True, generator=generator))) # generate 10 sentences with shuffling

Let us print the input ids.

In [45]:
input_ids

tensor([[  270,  1269,    22,  5484,    35,   394,     9,    67,  2949,    11,
            74,    63,   879,   289,    58,   322,     7,     6,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
        [   62,  2649,    11,   101,   883,    27,  6253,    25,    10,   296,
            11,    10,   564,     7,     6,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
    

In [47]:
input_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [48]:
labels

tensor([[  722,    79,     9,   224,  1993,  1501,    18,     9,   121,    34,
            14, 11443,   198,  2717,     7,     6,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
        [   90,  4722,  1634,   140,    65, 18011,    15,  1057,   738,   119,
             6,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
    

Let us decode the labels.

In [49]:
dataset.decode(labels)

['Xar mi, kenn toppatoowu ko, li muy foraatu lay dunde.',
 'Lafu séram dafa keppu ci buntu woto bi.',
 'Ndax',
 'Taawub baay, baay la fu baay nekkul.',
 'Ku tàbbal baag bi ci teen bi?',
 'Saxaar saa ngiy jolli bu baax.',
 'Ci fii!',
 'Dafay jénd ay mool di leen jaayaat.',
 'Moom daal, léegi addina dafa soppeeku.',
 'Na dem su bëggul!']

We will add the concatenation the input ids with the labels and also their concatenated attention masks in order to use them inside the GPT2 model. Notice that the concatenation of the inputs and the labels will be the new input ids and also the new labels.

In [50]:
# Initialize our custom dataset
dataset = SentenceDataset("data/extractions/new_data/corpora_v6.csv", wrapped_tokenizer1, truncation=True, decoder_only = True)

In [51]:
generator = torch.manual_seed(5)
input_ids, input_mask, labels, label_mask = next(iter(DataLoader(dataset, 10, shuffle=True, generator=generator))) # generate 10 sentences with shuffling

In [52]:
# new gpt2 input ids and also the new labels
input_sequence_gpt2 = torch.concat((input_ids, labels), axis = 1)

In [53]:
input_sequence_gpt2.shape

torch.Size([10, 124])

In [54]:
# new gpt2 mask attention
attention_mask_gpt2 = torch.concat((input_mask, label_mask), axis = 1)

In [55]:
attention_mask_gpt2.shape

torch.Size([10, 124])

In [59]:
dataset.decode(input_sequence_gpt2)

['Xar mi, kenn toppatoowu ko, li muy foraatu lay dunde.',
 'Lafu séram dafa keppu ci buntu woto bi.',
 'Ndax',
 'Taawub baay, baay la fu baay nekkul.',
 'Ku tàbbal baag bi ci teen bi?',
 'Saxaar saa ngiy jolli bu baax.',
 'Ci fii!',
 'Dafay jénd ay mool di leen jaayaat.',
 'Moom daal, léegi addina dafa soppeeku.',
 'Na dem su bëggul!']